In [8]:
pip install tensorflow


SyntaxError: invalid syntax (1141407339.py, line 1)

In [9]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from PIL import Image
from skimage.transform import resize

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from skimage.color import gray2rgb
import sys
np.set_printoptions(threshold=sys.maxsize, suppress=True)
# np.set_printoptions(suppress=True)
import os

ModuleNotFoundError: No module named 'distutils'

In [11]:
# load the model
model = ResNet50(weights='imagenet', include_top=False)

#summary
# print(model.summary())

#load pixel data from excel file
excel_file = '/Users/test/Sarcasm_Model_Dataset.xlsx'
df = pd.read_excel(excel_file)

numerical_values_list = []

for i in range(len(df)):
    frame1 = np.array(df['frame_1'][i].split()).astype(int)
    frame2 = np.array(df['frame_2'][i].split()).astype(int)
    frame3 = np.array(df['frame_3'][i].split()).astype(int)

    # Stack the frames along the last axis to create a single array
    stacked_frames = np.concatenate([frame1, frame2, frame3], axis=-1)

    # Reshape the stacked frames to match the input shape expected by ResNet50
    stacked_frames = stacked_frames.reshape((1, 48, 48, 3)) 

    # Preprocess the input frames
    preprocessed_frames = preprocess_input(stacked_frames)

    # Extract features using the model
    frame_features = model.predict(preprocessed_frames)
    flattened_features = frame_features.reshape(frame_features.shape[0], -1)

    # Convert float values to integers if the decimal part is zero
    flattened_features_int = np.where(np.mod(flattened_features, 1) == 0, flattened_features.astype(int), flattened_features)
    
    # Convert the numpy array to a string
    features_string = np.array2string(flattened_features_int, separator=',')    
    # print(features_string)
    
    #convert to numerical values
    numerical_values = eval(features_string)

    numerical_values_list.append(numerical_values[0])

df['visual_extract'] = numerical_values_list



NameError: name 'ResNet50' is not defined

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


X= df['visual_extract'] 
X = np.array(X.explode()).reshape(len(X), -1)

y=df['sarcasm'] 

X_train, X_test, y_train, y_test = train_test_split(X,y , 
                                   random_state=104,  
                                   test_size=0.25,  
                                   shuffle=True) 

model = RandomForestClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
score = accuracy_score(y_pred, y_test)
# print(score)

finalPrediction = []

for i in range(len(df)):
    outer = []
    prediction = []
    x_val = df['visual_extract'][i]
    outer.append(x_val)
    final_y = model.predict_proba(outer)
    result = final_y[0].reshape(1,-1).flatten().tolist()
    prediction.append(result[1])
    finalPrediction.append(prediction)

print(finalPrediction)